In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport Compute_Jacobian_Sioux

In [2]:
numNodes = Compute_Jacobian_Sioux.numNodes;
numLinks = Compute_Jacobian_Sioux.numLinks;
numZones = Compute_Jacobian_Sioux.numZones;
numODpairs = Compute_Jacobian_Sioux.numODpairs;
od_pairs = Compute_Jacobian_Sioux.od_pairs;
link_list_js = Compute_Jacobian_Sioux.link_list_js;
link_length_list = Compute_Jacobian_Sioux.link_length_list;

In [3]:
include("../Julia_files/initia_data.jl");
include("../Julia_files/load_network_uni_class.jl");
include("prepare_data_Sioux.jl");
include("extract_data_Sioux.jl");
include("../Julia_files/tap_MSA.jl");
include("../Julia_files/inverseVI.jl");
include("../Julia_files/demands_adjustment_gradi.jl");

In [4]:
demandsDiffDict[1] = norm(demandsDicToVec(demandsDict[1]) - demandsDicToVec(demandsDict[0]))/
                     norm(demandsDicToVec(demandsDict[0]));

In [5]:
fcoeffs = [1, 0, 0, 0, .15];

# get observed flow vector (corresponding to ground truth demands and ground truth costs)
tapFlowDicDict[0], tapFlowVecDict[0] = tapMSA(demandsDict[0], fcoeffs);

In [6]:
# get arcs data corresponding to ground truth demands and flows
arcsDict[0] = observFlow("./benchmark_data/Sioux_net.txt", tapFlowDicDict[0]);

In [7]:
deg = 6
c = 3.5
lam = 1.0

coeffs_dict[(deg, c, lam, 1)], objInvVIDict[1] = train(lam, deg, c, demandsDict[1], arcsDict[0]);

In [8]:
# get initial flow vector (corresponding to initial demands)
tapFlowDicDict[1], tapFlowVecDict[1] = tapMSA(demandsDict[1], coeffs_dict[(deg, c, lam, 1)]);

demandsVecDict[0] = demandsDicToVec(demandsDict[0]);

In [9]:
# weight parameters
gamma1 = 1
gamma2 = 1

1

In [10]:
demandsVecDict[1] = demandsDicToVec(demandsDict[1]);
objFunDict[1] = objF(gamma1, gamma2, demandsVecDict[1], demandsVecDict[1], coeffs_dict[(deg, c, lam, 1)]);

In [11]:
# get arcs data corresponding to initial demands and flows
arcsDict[1] = observFlow("./benchmark_data/Sioux_net.txt", tapFlowDicDict[1]);

In [12]:
linkCostDicDict[1] = tapFlowVecToLinkCostDict(tapFlowVecDict[1], coeffs_dict[(deg, c, lam, 1)]);

In [13]:
linkCostDicDict[1]["0"], link_length_list[1]

(6.0123802238228645,6.0)

In [14]:
jacobiSpiessDict[1] = Compute_Jacobian_Sioux.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                              link_list_js, [linkCostDicDict[1]["$(i)"] for i=0:numLinks-1]);

In [15]:
# maximum number of iterations
N = 100;  

# Armijo rule parameters
rho = 2;
M = 10;

# search direction parameter
epsilon_1 = 0;

# stop criterion parameter
epsilon_2 = 1e-20;

In [16]:
for l = 1:N
    
    jacobDict[l] = jacobiSpiessDict[l]
    
    demandsVecDict[l] = demandsDicToVec(demandsDict[l]);

    descDirecDict[l] = descDirec(gamma1, gamma2, demandsVecDict[l], demandsVecDict[1], tapFlowVecDict[l], 
        tapFlowVecDict[0], jacobDict[l], numODpairs, numLinks);

    searchDirecDict[l] = searchDirec(demandsVecDict[l], descDirecDict[l], epsilon_1);

    thetaMaxDict[l] = thetaMax(demandsVecDict[l], searchDirecDict[l]);

    demandsVecDict[l+1] = similar(demandsVecDict[0]);
    
    demandsVecDict[l+1], objFunDict[l+1] = armijo(gamma1, gamma2, objFunDict[l], demandsVecDict[l], 
        demandsVecDict[1], coeffs_dict[(deg, c, lam, l)], searchDirecDict[l], thetaMaxDict[l], rho, M);
            
    demandsDict[l+1] = demandsVecToDic(demandsVecDict[l+1]);
    
    tapFlowDicDict[l+1], tapFlowVecDict[l+1] = tapMSA(demandsDict[l+1], coeffs_dict[(deg, c, lam, l)]);
        
    arcsDict[l+1] = observFlow("./benchmark_data/Sioux_net.txt", tapFlowDicDict[l+1]);
        
    coeffs_dict[(deg, c, lam, l+1)], objInvVIDict[l+1] = train(lam, deg, c, demandsDict[l+1], arcsDict[0]);
    
#     objFunTmp = objF(gamma1, gamma2, demandsVecDict[l+1], demandsVecDict[1], 
#         coeffs_dict[(deg, c, lam, l+1)])
    
#     if objFunTmp > objFunDict[l+1]
#         coeffs_dict[(deg, c, lam, l+1)] = coeffs_dict[(deg, c, lam, l)]
#     end

    linkCostDicDict[l+1] = tapFlowVecToLinkCostDict(tapFlowVecDict[l+1], coeffs_dict[(deg, c, lam, l+1)]);
    
    jacobiSpiessDict[l+1] = Compute_Jacobian_Sioux.jacobianSpiess(numNodes, numLinks, numODpairs, od_pairs, 
                                              link_list_js, [linkCostDicDict[l+1]["$(i)"] for i=0:numLinks-1]);
        
    demandsDiffDict[l+1] = norm(demandsVecDict[l+1] - demandsVecDict[0]) / norm(demandsVecDict[0]);
    
    # stopping criterion
    if (objFunDict[l] - objFunDict[l+1]) / objFunDict[1] < epsilon_2
        break
    end
        
end

# normalize objective function value
for l = 1:(length(objFunDict))
    norObjFunDict[l] = objFunDict[l] / objFunDict[1];
end

In [17]:
demandsVecDict

Dict{Any,Any} with 23 entries:
  18 => [78.9311,107.187,479.285,206.652,256.876,513.084,829.132,450.373,1341.6…
  2  => [98.3441,105.194,473.829,209.182,276.134,522.699,837.626,466.148,1373.1…
  16 => [79.7778,107.38,478.791,206.344,257.436,513.786,828.976,450.387,1344.0,…
  11 => [81.7118,106.868,476.865,204.468,258.913,515.688,829.322,449.365,1347.1…
  21 => [78.985,107.317,479.648,207.033,257.084,513.526,829.74,450.697,1341.37,…
  0  => [100.0,100.0,500.0,200.0,300.0,500.0,800.0,500.0,1300.0,500.0  …  400.0…
  7  => [89.5716,107.596,477.17,207.05,266.762,520.13,833.544,456.337,1360.1,50…
  9  => [88.4929,107.349,476.865,206.343,265.64,519.343,832.807,455.01,1358.08,…
  10 => [88.01,107.318,476.862,206.154,265.181,519.095,832.584,454.528,1357.24,…
  19 => [78.9336,107.193,479.295,206.658,256.885,513.099,829.153,450.376,1341.5…
  17 => [78.8757,107.074,479.066,206.384,256.771,512.943,828.869,450.115,1341.5…
  8  => [88.8955,107.399,476.905,206.509,266.052,519.591,833.053,455.4,1358.74

In [18]:
norObjFunDict

Dict{Any,Any} with 22 entries:
  18 => 0.180609
  2  => 0.680624
  16 => 0.190828
  11 => 0.231972
  21 => 0.174241
  7  => 0.330612
  9  => 0.307892
  10 => 0.298771
  19 => 0.179309
  17 => 0.186215
  8  => 0.313475
  22 => 0.174241
  6  => 0.344652
  4  => 0.564348
  3  => 0.636088
  5  => 0.403301
  20 => 0.177916
  13 => 0.216307
  14 => 0.204918
  15 => 0.197353
  12 => 0.217888
  1  => 1.0

In [19]:
coeffs_dict

Dict{Any,Any} with 22 entries:
  (6,3.5,1.0,10) => [1.0,0.0106218,-0.0593991,0.0860574,0.103339,0.00815082,0.0…
  (6,3.5,1.0,17) => [1.0,0.0105049,-0.0588829,0.0852323,0.103933,0.00795426,0.0…
  (6,3.5,1.0,12) => [1.0,0.0105084,-0.0588984,0.0852569,0.103915,0.00796013,0.0…
  (6,3.5,1.0,9)  => [1.0,0.0106238,-0.0594078,0.0860715,0.103329,0.00815416,0.0…
  (6,3.5,1.0,2)  => [1.0,0.00827867,-0.0504685,0.0767698,0.106299,0.0083647,6.2…
  (6,3.5,1.0,20) => [1.0,0.0106073,-0.059335,0.0859549,0.103413,0.00812634,0.00…
  (6,3.5,1.0,7)  => [1.0,0.010654,-0.0596476,0.0868303,0.102503,0.00851754,0.00…
  (6,3.5,1.0,4)  => [1.0,0.0181877,-0.0847275,0.100304,0.104578,0.00561508,0.00…
  (6,3.5,1.0,16) => [1.0,0.0107239,-0.0598497,0.0867776,0.102822,0.00832227,0.0…
  (6,3.5,1.0,19) => [1.0,0.0105987,-0.059297,0.0858942,0.103457,0.00811188,0.00…
  (6,3.5,1.0,3)  => [1.0,0.00907216,-0.0529879,0.0776986,0.106601,0.00821198,5.…
  (6,3.5,1.0,6)  => [1.0,0.010273,-0.058287,0.0857901,0.102405,0.0088216,0.000

In [20]:
outfile = open("./results/norObjFunDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, norObjFunDict)

close(outfile)

In [21]:
outfile = open("./results/demandsDiffDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsDiffDict)

close(outfile)

In [22]:
outfile = open("./results/objInvVIDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, objInvVIDict)

close(outfile)

In [23]:
outfile = open("./results/coeffs_dict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, coeffs_dict)

close(outfile)

In [24]:
outfile = open("./results/demandsVecDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsVecDict)

close(outfile)

In [25]:
outfile = open("./results/demandsDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, demandsDict)

close(outfile)

In [26]:
outfile = open("./results/tapFlowDicDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, tapFlowDicDict)

close(outfile)

In [27]:
outfile = open("./results/tapFlowVecDict_$(gamma1)_$(gamma2).json", "w")

JSON.print(outfile, tapFlowVecDict)

close(outfile)